In [132]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [133]:
from machine import I2C, Pin

i2c = I2C(scl=Pin(15), sda=Pin(4))
print(i2c.scan())

[60, 83]


In [134]:
print("DEVID %X (should be E5)" % i2c.readfrom_mem(0x53, 0x00, 1)[0])

DEVID E5 (should be E5)


In [145]:
i2c.writeto_mem(0x53, 0x2E, b"\x00")  # INT_ENABLE    (disable)

i2c.writeto_mem(0x53, 0x2A, b"\x05")  # TAP_AXES      (X+Z)  (not z)
i2c.writeto_mem(0x53, 0x1d, b"\x20")  # THRESH_TAP    (*g/16 tap acceleration)
i2c.writeto_mem(0x53, 0x21, b"\x30")  # DUR           (*625us duration of tap)
i2c.writeto_mem(0x53, 0x22, b"\x40")  # LATENT        (*1.25ms earliest second tap)
i2c.writeto_mem(0x53, 0x23, b"\xFF")  # WINDOW        (*1.25ms latest second tap-LATENT)

i2c.writeto_mem(0x53, 0x24, b"\x10")  # THRESH_ACT    (*g/16 acceleration)
i2c.writeto_mem(0x53, 0x25, b"\x08")  # THRESH_INACT  (*g/16 acceleration)
i2c.writeto_mem(0x53, 0x26, b"\x05")  # TIME_INACT    (*1secs)
i2c.writeto_mem(0x53, 0x27, b"\x55")  # ACT_INACT_CTL (DirectComp+X+Z ACT+INACT)

i2c.writeto_mem(0x53, 0x28, b"\x07")  # THRESH_FF     (*g/16 RMS FreeFall)
i2c.writeto_mem(0x53, 0x29, b"\x07")  # TIME_FF       (*g/16 RMS FreeFall)

#i2c.writeto_mem(0x53, 0x2D, b"\x04")  # POWER_CTL   (SLEEP+8HZ)
#i2c.writeto_mem(0x53, 0x2D, b"\x08")  # POWER_CTL     (MEASURE)
i2c.writeto_mem(0x53, 0x2D, b"\x18")  # POWER_CTL     (autosleep+MEASURE)

i2c.writeto_mem(0x53, 0x31, b"\x0A")  # DATA_FORMAT   (FULL_RES+8g)
#i2c.writeto_mem(0x53, 0x2E, b"\x50")  # INT_ENABLE    (SINGLE_TAP+activity)
i2c.writeto_mem(0x53, 0x2E, b"\x48")  # INT_ENABLE    (SINGLE_TAP+inactivity)
i2c.writeto_mem(0x53, 0x2F, b"\x40")  # INT_MAP       (SINGLE_TAP=INT2)


In [146]:
irqp12 = 0
irqp13 = 0
def c12(v):
    global irqp12
    irqp12 += 1
    y = i2c.readfrom_mem(0x53, 0x30, 1)

def c13(v):
    global irqp13
    irqp13 += 1
    y = i2c.readfrom_mem(0x53, 0x30, 1)
    
p12 = Pin(12, Pin.IN)
p12.irq(c12, Pin.IRQ_RISING)
p13 = Pin(13, Pin.IN)
p13.irq(c13, Pin.IRQ_RISING)


In [147]:
import ustruct, time
for i in range(20):
    x = i2c.readfrom_mem(0x53, 0x32, 6)
    print(irqp12, irqp13, ustruct.unpack("<hhh", x))
    time.sleep(4)



0 0 (2, 260, 52)
1 3 (3, 261, 49)
.1 6 (2, 259, 44)
.2 6 (2, 262, 42)
2 6 (3, 261, 44)
.3 6 (3, 260, 44)
.4 6 (165, 194, 98)
4 48 (2, 260, 45)
.5 56 (3, 260, 50)
.5 64 (4, 257, 66)
.5 66 (4, 259, 64)
5 96 (-58, 259, 211)
.5 106 (2, 259, 66)
.6 106 (3, 259, 66)


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
KeyboardInterrupt: 


In [118]:
y = i2c.readfrom_mem(0x53, 0x30, 1)
print(y)

b'\xc3'


In [164]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [165]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext

i2c.writeto_mem(0x53, 0x2E, b"\x00")  # INT_ENABLE    (disable)

i2c.writeto_mem(0x53, 0x2A, b"\x05")  # TAP_AXES      (X+Z)  (not z)
i2c.writeto_mem(0x53, 0x1d, b"\x20")  # THRESH_TAP    (*g/16 tap acceleration)
i2c.writeto_mem(0x53, 0x21, b"\x30")  # DUR           (*625us duration of tap)
i2c.writeto_mem(0x53, 0x22, b"\x40")  # LATENT        (*1.25ms earliest second tap)
i2c.writeto_mem(0x53, 0x23, b"\xFF")  # WINDOW        (*1.25ms latest second tap-LATENT)

i2c.writeto_mem(0x53, 0x24, b"\x10")  # THRESH_ACT    (*g/16 acceleration)
i2c.writeto_mem(0x53, 0x25, b"\x08")  # THRESH_INACT  (*g/16 acceleration)
i2c.writeto_mem(0x53, 0x26, b"\x05")  # TIME_INACT    (*1secs)
i2c.writeto_mem(0x53, 0x27, b"\x55")  # ACT_INACT_CTL (DirectComp+X+Z ACT+INACT)

i2c.writeto_mem(0x53, 0x28, b"\x07")  # THRESH_FF     (*g/16 RMS FreeFall)
i2c.writeto_mem(0x53, 0x29, b"\x07")  # TIME_FF       (*g/16 RMS FreeFall)

#i2c.writeto_mem(0x53, 0x2D, b"\x04")  # POWER_CTL   (SLEEP+8HZ)
#i2c.writeto_mem(0x53, 0x2D, b"\x08")  # POWER_CTL     (MEASURE)
i2c.writeto_mem(0x53, 0x2D, b"\x18")  # POWER_CTL     (autosleep+MEASURE)

i2c.writeto_mem(0x53, 0x31, b"\x0A")  # DATA_FORMAT   (FULL_RES+8g)
#i2c.writeto_mem(0x53, 0x2E, b"\x50")  # INT_ENABLE    (SINGLE_TAP+activity)
i2c.writeto_mem(0x53, 0x2E, b"\x48")  # INT_ENABLE    (SINGLE_TAP+inactivity)
i2c.writeto_mem(0x53, 0x2F, b"\x40")  # INT_MAP       (SINGLE_TAP=INT2)


import ustruct, time
for i in range(2000):
    y = i2c.readfrom_mem(0x53, 0x30, 1)[0]
    x = i2c.readfrom_mem(0x53, 0x32, 6)
    if (y&0x40):
        print("Tap")
        fbuff.fill(0)
        fbuff.text("Moo!", 0, 0, 1)
        doublepixels()
        doublepixels()
        oledshow()
        time.sleep(1)
        fbuff.fill(0)
        oledshow()
    elif (y&0x08):
        print("inactivity")
        print(ustruct.unpack("<hhh", x))
        fbuff.fill(0)
        fbuff.text("sleeping", 0, 0, 1)
        doublepixels()
        oledshow()
    time.sleep(0.1)


Sent 50 lines (2046 bytes) to main.py.
